In [ ]:
import pandas as pd
import requests
from io import StringIO
%conda install lxml #comment this out after done
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
    
url = "https://en.wikipedia.org/wiki/List_of_footballers_with_500_or_more_goals"
res = requests.get(url, headers=headers).text
res = StringIO(res) #pandas read_html needs a file-like object, so we use StringIO to convert the string to a file-like object


df = pd.read_html(res)
df

2 channel Terms of Service accepted
Retrieving notices: done
Channels:
 - defaults
Platform: osx-64

NoSpaceLeftError: No space left on devices.


Note: you may need to restart the kernel to use updated packages.


ImportError: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.

In [ ]:
df = df[3]

In [ ]:
df.to_csv("./data/top500goals.csv", index=False)
#comment above out when done

In [ ]:
df = pd.read_csv("./data/footballers.csv")
df.head()

In [ ]:
import random, math

df["Career span"] = df["Career span"].str.split("—").str[0]

In [ ]:
df["Career span"].isna().sum()

In [ ]:
df.loc[~df['Goals'].str.isnumeric()] #comment out after running
df["Goals"] = [item.replace("+", "") for item in df["Goals"]]
df["Goals"]

In [ ]:
df["Goals"] = df["Goals"].astype(int)
df["Career span"] = df["Career span"].astype(int)
df.head()

In [ ]:
df["Matches"] = df["Matches"].str.replace("+", "").astype(int)
df.describe()

In [ ]:
from datetime import datetime, timedelta
rows = []
columns = ["name", "date", "goals", "assists"]

for row in df.itertuples():
    name = row.Player
    start_date = f"{row._6}-08-01"
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    goal_ratio = row.Ratio
    goal_sd = random.random()
    goal_sd = goal_sd if goal_sd < 0.7 else 0.7
    assist_sd = 1 - goal_sd
    assist_ratio = 1- goal_ratio
    
    for i in range(row.Matches):
        name = name
        match_date = timedelta(days=3 * i) + start_date
        goals = random.normalvariate(goal_ratio, goal_sd)
        assists = random.normalvariate(assist_ratio, assist_sd)
        goals = round(goals) if goals > 0 else 0
        assists = round(assists) if assists > 0 else 0
        row = [name, match_date, goals, assists]
        rows.append(row)
games_df = pd.DataFrame(data=rows, columns=columns)
games_df.head()

In [ ]:
games_df.shape

In [ ]:
games_df.sample(10) #random sample of 10 rows from the dataframe

In [ ]:
fake_df = games_df.groupby("name").agg(
    matches=("name", "count"),
    goals=("goals", "sum"),
    assists=("assists", "sum"),
    g_ratio=("goals", "mean"),
    a_ratio=("assists", "mean")  
).reset_index().sort_values("goals", ascending=False).reset_index(drop=True)

fake_df.head(10)

In [ ]:
for i in range(10):
    print(f"{i} df: {df.loc[i, "Player", "Goals"]} | {i}: fake {fake_df.loc[i,["name", "goals"]]}")